In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

In [ ]:
nEvents = 100000
runIDs = arr([184])

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, nEvents*4)
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    
M = runs[runIDs[0]]["M"]

In [ ]:
runID = 184
shareCoords = False
fig, axs = plt.subplots(1, 1, figsize=(8, 6), sharex=shareCoords, sharey=shareCoords)

binN = 20
binRange = [0,300]
# binRange = None
Name = "Falltime"
column = M[Name]

# mask = np.any(runs[runIDs[0]]["data"][:,:,M["Charge"]] > 2000, axis=1)
pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)

for i_pix in range(4):
    # mask = runs[runIDs[0]]["data"][:,i_pix,M["Charge"]] >= 220
    # print(mask)
    # mask = i_pix == pixWithHighestCharge
    entries = runs[runIDs[0]]["data"][:,i_pix, column]
    entries = entries[entries != 0]
    hist, bins = np.histogram(entries, bins=binN, range=binRange)
    
    Hist.draw(axs, hist, bins, color="C"+str(i_pix), fill_alpha=0.1, label="Pix "+str(i_pix+1)+" (N={:.1f}k)".format(np.sum(hist)/1000))
    
axs.set_xlim(binRange)
Hist.finalize(runs[runID], fig, axs, Name, "Entries", Name, measurement="TB", logy=False)

In [ ]:
runID = 184

rangeX = [0, 50]
rangeY = [0, 300]
binNX = 20
binNY = 20
NameX = "Risetime"
NameY = "Falltime"
shareCoords = True
logScale = True

colX = M[NameX]
colY = M[NameY]
entriesX = runs[runID]["data"][:,i_pix,colX]
entriesY = runs[runID]["data"][:,i_pix,colY]

pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)


# hist, binsx, binsy = np.histogram2d(a, b)

# fig, axs = plt.subplots(2,2, figsize=(6, 4))

fig, axs = Hist.create_fig(2,2, [8,6], sharex=shareCoords, sharey=shareCoords)

for i_pix in range(4):
	# mask = i_pix == pixWithHighestCharge
	mask = len(runs[runID]["data"][:,i_pix,colX])*[True]
	# mask = np.logical_and(runs[runID]["data"][mask,i_pix,colX] != 0, runs[runID]["data"][mask,i_pix,colY] != 0) # remove 0 entries
	# mask = runs[runID]["data"][mask,i_pix,colY] != 0 # remove 0 entries
	entriesX = runs[runID]["data"][mask,i_pix,colX]
	entriesY = runs[runID]["data"][mask,i_pix,colY]
	hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])

	extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]

	if not logScale:
		axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
	else:
		axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', norm=LogNorm(), cmap='viridis')

# lt.imshow(H, interpolation='nearest', origin='lower',

        # extent=[xedges[0], xedges[-1], yedges[0, yedges[-1]])

Hist.finalize(runs[runID], fig, axs, title=NameY+" vs "+NameX, xlabel=NameX, ylabel=NameY)

In [ ]:
runID = 184

fig, axs = Hist.create_fig(2,2, [8,6])
column = M["ToT"]
print(column)
binN = 50
binRange = [0,0.5]

for i_pix in range(4):
    mask = np.any(runs[runID]["data"][:,i_pix,M["Charge"]] > 220)
    entries = runs[runID]["data"][:,i_pix,column]
    hist, bins = np.histogram(entries, bins=binN, range=binRange)
    print(entries)
    Hist.draw(axs[i_pix], hist, bins, color="C"+str(i_pix), fill_alpha=0.1, label="Pix "+str(i_pix+1)+" (N={:.1f}k)".format(np.sum(hist)/1000))


In [ ]:
class Hist_2D:
    def __init__(self, binsX, binsY):
        self.binsX = arr(binsX)
        self.binsY = arr(binsY)
        self.val = np.zeros((len(binsX)+1,len(binsY)+1))
        self.err = np.zeros((len(binsX)+1,len(binsY)+1))
        
    def __init__(self, binNX, binNY, rangeX, rangeY):
        self.binsX = np.linspace(rangeX[0],rangeX[1],binNX+2)
        self.binsY = np.linspace(rangeY[0],rangeY[1],binNY+2)
        self.val = np.zeros((binNX,binNY))
        self.err = np.zeros((binNX,binNY))
    
    def setBin(self, binX, binY, value): 
        self.histo[binX+1,binY+1] = value
    
    def fill(self, x, y, w=1):
        if x < self.binsX[1]:
            if y < self.binsY[1]:
                self.val[0,0] += w
                self.err[0,0] += w**2
            elif y > self.binsY[-2]:
                self.val[0,-1] += w
                self.err[0,-1] += w**2
            else:
                iY = np.digitize(y, self.binsY)-1
                self.val[0,iY] += w
                self.err[0,iY] += w**2
        elif x > self.binsX[-2]:
            if y < self.binsY[1]:
                self.val[-1,0] += w
                self.err[-1,0] += w**2
            elif y > self.binsY[-2]:
                self.val[-1,-1] += w
                self.err[-1,-1] += w**2
            else:
                iY = np.digitize(y, self.binsY)-1
                self.val[-1,iY] += w
                self.err[-1,iY] += w**2
        else:
            iX = np.digitize(x, self.binsX)-1
            if y < self.binsY[1]:
                self.val[iX,0] += w
                self.err[iX,0] += w**2
            elif y > self.binsY[-2]:
                self.val[iX,-1] += w
                self.err[iX,-1] += w**2
            else:
                iY = np.digitize(y, self.binsY)-1
                self.val[iX,iY] += w
                self.err[iX,iY] += w**2
                
        iX = np.digitize(x, self.binsX)-1
        iY = np.digitize(y, self.binsY)-1
        self.val[iX,iY] += w
        self.err[iX,iY] += w**2
        
    def hist(self):
        return self.val[1:-2, 1:-2]
    def histErr(self):
        return self.err[1:-2, 1:-2]
    
    def rangeX(self):
        return self.binsX[1:-2]
    def rangeY(self):
        return self.binsY[1:-2]
    def range(self):
        return self.binsX[1:-2], self.binsY[1:-2]
    
    def binNX(self):
        return len(self.binsX)-2
    def binNY(self):
        return len(self.binsY)-2
    def binN(self):
        return self.binNX()*self.binNY()